In [1]:
import json
import os

images = 'images'
labels = 'labels'

path = f'D:/상하수도 데이터(하수관로 내부 이미지)/Training/{images}'

files_and_folders = os.listdir(path)

class_folders = [folder for folder in files_and_folders if os.path.isdir(os.path.join(path, folder))]

for folder in class_folders:
    print(folder)

print('------------------------')
print(f'class(범주)개수 : {len(class_folders)}\n')   # 라벨링데이터_압축 폴더, 하수관로 내부 폴더 제거

토사퇴적(Deposits-Silty,DS)
균열-원주(Crack-Circumferential,CC)
표면손상(Surface-Damage,SD)
파손(Broken-Pipe,BK)
이음부-손상(Joint-Faulty,JF)
균열-길이(Crack-Longitudinal,CL)
연결관-돌출(Lateral-Protruding,LP)
이음부-단차(Joint-Displaced,JD)
------------------------
class(범주)개수 : 8



<h3> image에 해당하는 annotation연결 및 txt파일로 정리

In [28]:
from glob import glob
import pandas as pd
import json2table
import numpy as np
import making_txt

for i in range(len(class_folders)):

    class_number = i
    selected_class = class_folders[class_number]

    with open(f'D:/상하수도 데이터(하수관로 내부 이미지)/Training/{labels}/{selected_class}.json', 'r', encoding='UTF8') as file:
        data = json.load(file)

    img_path_list = glob(f'D:/상하수도 데이터(하수관로 내부 이미지)/Training/{images}/{selected_class}/*')
    img_list = [os.path.basename(img_path) for img_path in img_path_list]

    df = json2table.open_optimized(data, img_list)

    df = df[(df["x"] <= 1) & (df["y"] <= 1) & (df["w"] <= 1) & (df["h"] <= 1)]
    
    random_numbers = np.random.permutation(df['image_id'].unique())
    selected_numbers = random_numbers[:2000]
    df = df[df['image_id'].isin(selected_numbers)]

    making_txt.df2txt(df, data, selected_class, class_number)

Processing: 100%|██████████| 6241/6241 [00:31<00:00, 197.90it/s]


<h3> image파일과 그에 맞는 bbox정보를 담고있는 txt파일 연결

In [29]:
def copy(path, changed_path, file_name, category='_'):
    
    import shutil
    import os

    if category == 'images':
        shutil.copy(os.path.join(path, file_name), os.path.join(changed_path, 'images', file_name))
    
    elif category == 'labels':
        shutil.copy(os.path.join(path, file_name), os.path.join(changed_path, 'labels', file_name))

In [34]:
# import random
# import os
# from tqdm import tqdm

# def split_dataset(image_folder, txt_folder, train_folder, valid_folder, test_folder, split_ratio=(1.0, 0, 0)):
#     # 이미지 파일 목록, 텍스트 파일 목록 호출
#     image_files = os.listdir(image_folder)                                          # image 목록
#     text_files = os.listdir(txt_folder)                                             # txt 목록

#     # 이미지 파일명과 텍스트 파일명 연결
#     file_mapping = {name.split('.')[0] : name for name in text_files}              # file_mapping = {text_name : image_file}의 형식으로 저장
    
    
#     # 이미지 파일을 랜덤하게 섞음
#     shuffled_images = list(file_mapping.keys())
#     random.shuffle(shuffled_images)

#     # 데이터셋 길이와 분할 비율 계산
#     num_images = len(shuffled_images)
#     train_split = int(num_images * split_ratio[0])
#     valid_split = int(num_images * split_ratio[1])

#     # 폴더 생성
#     for folder in [train_folder, valid_folder, test_folder]:
#         os.makedirs(os.path.join(folder, 'images'), exist_ok=True)
#         os.makedirs(os.path.join(folder, 'labels'), exist_ok=True)
    
#     # 이미지 파일 및 대응하는 텍스트 파일 일괄 복사
#     with tqdm(total = num_images, desc = 'Copying Files') as pbar:
#         for i, (text_name, image_file) in enumerate(file_mapping.items()):
#             text_file = f'{text_name}.txt'
            

#             if i < train_split:
            
#                 copy(image_folder, train_folder, image_file, category='images')
#                 if text_file in text_files:
#                     copy(txt_folder, train_folder, text_file, category='labels')


#             elif train_split <= i < train_split + valid_split:

#                 copy(image_folder, train_folder, image_file, category='images')
#                 if text_file in text_files:
#                     copy(txt_folder, train_folder, text_file, category='labels')
            
#             else:

#                 copy(image_folder, train_folder, image_file, category='images')
#                 if text_file in text_files:
#                     copy(txt_folder, train_folder, text_file, category='labels')
            
#             pbar.update(1)

In [36]:
# 변경안

import random
import os
from tqdm import tqdm

def split_dataset(image_folder, txt_folder, train_folder, valid_folder, test_folder, split_ratio=(1.0, 0, 0)):
    # 이미지 파일 목록, 텍스트 파일 목록 호출
    image_files = os.listdir(image_folder)                                          # image 목록
    text_files = os.listdir(txt_folder)                                             # txt 목록

    # 이미지 파일명과 텍스트 파일명 연결
    file_mapping = {name.split('.')[0] : name for name in text_files}              # file_mapping = {text_name : image_file}의 형식으로 저장
    
    
    # 이미지 파일을 랜덤하게 섞음
    shuffled_images = list(file_mapping.keys())
    random.shuffle(shuffled_images)

    # 데이터셋 길이와 분할 비율 계산
    num_images = len(shuffled_images)
    train_split = int(num_images * split_ratio[0])
    valid_split = int(num_images * split_ratio[1])

    # 폴더 생성
    for folder in [train_folder, valid_folder, test_folder]:
        os.makedirs(os.path.join(folder, 'images'), exist_ok=True)
        os.makedirs(os.path.join(folder, 'labels'), exist_ok=True)
    
    # 이미지 파일 및 대응하는 텍스트 파일 일괄 복사
    with tqdm(total = num_images, desc = 'Copying Files') as pbar:
        for i, (img_name, bbox_file) in enumerate(file_mapping.items()):
            img_file = f'{img_name}.png'
            

            if i < train_split:
            
                copy(image_folder, train_folder, img_file, category='images')
               #copy해라image_folder에서train_folder로img_file을
                if bbox_file in text_files:
                    copy(txt_folder, train_folder, bbox_file, category='labels')


            elif train_split <= i < train_split + valid_split:

                copy(image_folder, train_folder, img_file, category='images')
                if bbox_file in text_files:
                    copy(txt_folder, train_folder, bbox_file, category='labels')
            
            else:

                copy(image_folder, train_folder, img_file, category='images')
                if bbox_file in text_files:
                    copy(txt_folder, train_folder, bbox_file, category='labels')
            
            pbar.update(1)

In [37]:
for folder in class_folders:
    # 이미지, 텍스트 파일이 있는 폴더 경로 설정
    image_folder_path = f'D:/상하수도 데이터(하수관로 내부 이미지)/Training/images/{folder}'
    text_folder_path = f'D:/sewer detect/folder/{folder}_folder'

    # 결과를 저장할 train, valid, test 폴더 경로 설정
    train_folder_path = 'D:/train'
    valid_folder_path = 'D:/valid'
    test_folder_path = 'D:/test'

    # 폴더 생성 및 데이터셋 분할
    split_dataset(image_folder_path, text_folder_path, train_folder_path, valid_folder_path, test_folder_path, split_ratio=(0, 1.0, 0))

Copying Files: 100%|██████████| 6000/6000 [35:21<00:00,  2.83it/s]  
